***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [1]:
import numpy as np
import chart_studio
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

C:\Users\Sif\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



<div class="alert alert-block alert-info">
<b>Load the stock historical data, transform in montly data and get percentage return</b>
</div>

In [2]:
#For a brazilian stock as FLRY3 you have to put a .SA so it will be FLRY3.SA

R = data.DataReader('SNE', 'yahoo', start = '2016/12/01', end = '2019/12/31')
R = R.resample('M').last().pct_change()
R.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-31,0.080725,0.078467,0.080714,0.079914,0.659073,0.079914
2017-02-28,0.025008,0.026246,0.025777,0.022795,-0.227816,0.022795
2017-03-31,0.084430,0.087407,0.086018,0.089470,-0.217370,0.092539
2017-04-30,0.023979,0.017267,0.018985,0.023421,0.509737,0.023421


<div class="alert alert-block alert-info">
<b>Percentage return of the market index S&P 500 with GSPC</b>
</div>

In [3]:
Rm = data.DataReader('^GSPC', 'yahoo', start = '2016/12/01', end = '2019/12/31')
Rm = Rm.resample('M').last().pct_change()
Rm.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-12-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-31,0.011320,0.015038,0.009953,0.017884,0.530364,0.017884
2017-02-28,0.038919,0.040468,0.040483,0.037198,0.030016,0.037198
2017-03-31,0.001081,0.001543,-0.000533,-0.000389,-0.203326,-0.000389
2017-04-30,0.009842,0.008364,0.012204,0.009091,0.108571,0.009091


<div class="alert alert-block alert-info">
<b>Percentage return of risk free investment with DGS10 = One Month Treasury Constant Maturity Rate</b>
</div>

In [4]:
Rf = data.DataReader('DGS1MO', 'fred', start = '2017/01/01', end = '2019/12/31')
Rf = Rf.resample('M').last()
Rf = Rf/100
Rf.head()

,DGS1MO
DATE,
2017-01-31,0.0050
2017-02-28,0.0040
2017-03-31,0.0074
2017-04-30,0.0068
2017-05-31,0.0086


<div class="alert alert-block alert-info">
<b>Table for Capital Asset Pricing Model</b>
</div>

In [5]:
df = pd.DataFrame({'R': R['Adj Close'], 'Rm': Rm['Adj Close'], 'Rf': Rf['DGS1MO']}).dropna()
df['Rm - Rf'] = df['Rm'] - df['Rf']
df.head()

,R,Rm,Rf,Rm - Rf
Date,,,,
2017-01-31,0.079914,0.017884,0.0050,0.012884
2017-02-28,0.022795,0.037198,0.0040,0.033198
2017-03-31,0.092539,-0.000389,0.0074,-0.007789
2017-04-30,0.023421,0.009091,0.0068,0.002291
2017-05-31,0.060834,0.011576,0.0086,0.002976


<div class="alert alert-block alert-info">
<b>Linear regression (CAPM) for expected returns</b>
</div>

In [6]:
x = df['Rm - Rf']
y = df['R']

x = sm.add_constant(x)
model = sm.OLS(y, x, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      R   R-squared:                       0.284
Model:                            OLS   Adj. R-squared:                  0.263
Method:                 Least Squares   F-statistic:                     13.48
Date:                Mon, 21 Sep 2020   Prob (F-statistic):           0.000820
Time:                        10:52:05   Log-Likelihood:                 51.430
No. Observations:                  36   AIC:                            -98.86
Df Residuals:                      34   BIC:                            -95.69
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0331      0.010      3.296      0.002       0.013       0.054
Rm - Rf        1.0207      0.278      3.672      0.001       0.456       1.586
==============================================================================
Omnibus:                        0.816   Durbin-Watson:                   2.197
Prob(Omnibus):                  0.665   Jarque-Bera (JB):                0.163
Skew:                           0.050   Prob(JB):                        0.922
Kurtosis:                       3.314   Cond. No.                         28.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Main Table</b>
</div>

In [7]:
# Predicted values for required return of stock(R)
y_pred = results.predict()
df['Predicted required returns'] = y_pred

# Simple residuals
residual = results.resid
df['Residuals'] = residual

# Squared residuals
squared_res = df['Residuals']**2
df['Squared residuals'] =  squared_res

# Sum of squared residuals
sum_squared_res = df['Squared residuals'].sum()
print('Sum of Squared residual: %f' %sum_squared_res)

df.head()

Sum of Squared residual: 0.121046


,R,Rm,Rf,Rm - Rf,Predicted required returns,Residuals,Squared residuals
Date,,,,,,,
2017-01-31,0.079914,0.017884,0.0050,0.012884,0.046282,0.033633,0.001131
2017-02-28,0.022795,0.037198,0.0040,0.033198,0.067016,-0.044222,0.001956
2017-03-31,0.092539,-0.000389,0.0074,-0.007789,0.025180,0.067359,0.004537
2017-04-30,0.023421,0.009091,0.0068,0.002291,0.035469,-0.012047,0.000145
2017-05-31,0.060834,0.011576,0.0086,0.002976,0.036168,0.024666,0.000608


<div class="alert alert-block alert-info">
<b>Expected monthly % returns plot</b>
</div>

In [9]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['Rm - Rf'], y = df['R'], 
    name = 'Observed required returns', 
    mode = 'markers',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(66, 133, 244)')))

fig.add_trace(go.Scatter(x = df['Rm - Rf'], y = df['Predicted required returns'],
    name = 'Predicted required returns',
    mode = 'lines',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(234, 67, 53)')))

fig.update_layout(title_text = 'Sony (SNE) and S&P 500 Index from 2017 to 2019', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(
    x = 0, y = 1.5),
    xaxis_title = 'Monthly market risk premium',
    yaxis_title = 'Monthly SNE required return',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Linear regression of monthly required return - Sony (SNE) and S&P 500', auto_open = False)

$$R\ =\ R_f\ +\ \beta(R_m\ -\ R_f)$$